<a href="https://colab.research.google.com/github/goongs/beautiful/blob/main/beautiful_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import urllib.request
from urllib.parse import quote
import pandas as pd
from collections import Counter
from konlpy.tag import Mecab

def get_movie_reviews(mcode, page_num=10):
  
  movie_review_df = pd.DataFrame(columns=("Title","Score","Review"))
  url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword="+str(mcode)+"&target=after"
  idx = 0
  for _ in range(0, page_num):
    movie_page = urllib.request.urlopen(url).read()
    movie_page_soup = BeautifulSoup(movie_page, 'html.parser')

    review_list = movie_page_soup.find_all('td',{'class':'title'})
    for review in review_list:
      title = review.find('a', {'class':'movie color_b'}).get_text()
      score = review.find('em').get_text()
      review_text = review.find('a',{'class':'report'}).get('onclick').split(',')[2]
      movie_review_df.loc[idx] = [title, score, review_text]
      idx += 1
      print("#",end="")
    try:
      url = "https://movie.naver.com/" + movie_page_soup.find('a',{'class':'pg_next'}).get('href')
    except:
     
      break

  return movie_review_df
def get_meanscore(mcode):
  url = "https://movie.naver.com/movie/point/af/list.naver?st=mcode&sword="+str(mcode)+"&target=after"
  html = urllib.request.urlopen(url).read()
  soup = BeautifulSoup(html, 'html.parser')
  meanscore1 = soup.find('table',{'class':'info_area'})
  meanscore2 = meanscore1.find('strong').get_text()[0:3]
  meanscore=float(meanscore2)
  return meanscore

In [ ]:
url = "https://movie.naver.com/movie/point/af/list.naver"
naver_movie = urllib.request.urlopen(url).read()
soup = BeautifulSoup(naver_movie, 'html.parser')
select = soup.find('select',{'id' : 'current_movie'})
movies = select.find_all('option')
meanscore1 = {}
idx1=0
movies_dict = {}
for movie in movies[1:]:
  movies_dict[movie.get('value')] = movie.get_text()

movie_review_df = pd.DataFrame(columns=("Title","Score","Review"))
for mcode in movies_dict:
  df = get_movie_reviews(mcode, 5)
  movie_review_df = pd.concat([movie_review_df, df])
  meanscore1[idx1]=get_meanscore(mcode)
  idx1+=1



In [ ]:
movie_mean_score = pd.DataFrame(columns=("Title","meanscore"))
url = "https://movie.naver.com/movie/point/af/list.naver"
soup1 = BeautifulSoup(naver_movie, 'html.parser')
select = soup.find('select',{'id' : 'current_movie'})
movies = select.find_all('option')
movies_dict = {}
for movie in movies[1:]:
  movies_dict[movie.get('value')] = movie.get_text()
movies_list=list(movies_dict.values())
movie_mean_score["Title"]=movies_list
score_list=list(meanscore1.values())
movie_mean_score["meanscore"] = score_list
stopword_words = "영화 전 난 일 걸 뭐 줄 만 건 분 개 끝 잼 이거 번 중 듯 때 게 내 말 나 수 거 점 것 제 기"
stopword_words = stopword_words.split(' ')















In [ ]:
def favorite(a):
    fs = movie_mean_score.drop([], axis=0)
    de = movie_mean_score[movie_mean_score["meanscore"] < a].index
    de1 = movie_mean_score.drop(de)
    return de1


def keyword(b):
    pp = movie_review_df[movie_review_df['Title'] == b]
    review2 = pp['Review']
    review3 = []
    review5 = []
    for i in range(0, 9):
        review3 += str(review2[i])
    review4 = str.join('', review3)
    review5 = review4.split(' ')
    tagger = Mecab()

    nouns = []
    for review6 in review5:
      for noun in tagger.nouns(review6):
        if noun not in stopword_words:
          nouns.append(noun)
    nouns_counter = Counter(nouns)
    top_nouns = dict(nouns_counter.most_common(5))
    return top_nouns
  



  



In [ ]:

print('평점 기대치:')
a=float(input())
fas=favorite(a)
print(fas)
print('영화 선택:')
b=input()
topiclist={}
topiclist=keyword(b)
print(topiclist)








In [ ]:
movie_mean_score

In [ ]:
movie_review_df